In [1]:
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()
# 2021.10.08 데이터 추출 완료 - 31140 데이터가 대부분 없음, InterfaceError: Python type ResultSet cannot be converted > updowncd 값 2에 의한 에러
cursor = mydb.cursor(prepared=True)
citycode = '36030'

query1 = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query1)

location_list1 = []

for a in cursor:
    location_list1.append(a)

now = datetime.now()
nowDate = now.strftime('%Y%m%d')

b_cnt = 0
for z in location_list1:
    if b_cnt == 1:
        break
        
    location = z
    query2 = ("select distinct routeid from public_busroute_info where citycode = '{}'".format(location[0]))
    cursor.execute(query2)

    location_list2 = []

    for b in cursor:
        location_list2.append(b)

    cnt = 1
    for i in location_list2:
        routeId = i

        characters = "(',')"

        join_data = ''.join( x for x in routeId if x not in characters)
        print("조회날짜 : {} | location : {} | cnt : {} | start {} public_busroute_info".format(nowDate,location[0],cnt,join_data))

        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteAcctoThrghSttnList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('pageNo') : '1',
            quote_plus('numOfRows') : '10000',
            quote_plus('routeId') : join_data
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],join_data))
            mydb.close()
            b_cnt += 1
            break
        if rows == []:
            print("location {} | {} 데이터가 없습니다.".format(location[0],join_data))
            cnt += 1
            continue
        cnt += 1
        for j in rows:
            item = j
            lat = item.findAll('gpslati')                
            lng = item.findAll('gpslong')
            nodeid = item.findAll('nodeid')
            nodenm = item.findAll('nodenm')
            nodeno = item.findAll('nodeno')
            if nodeno == []:
                nodeno = None
            else:
                nodeno = nodeno[0].text
            nodeord = item.findAll('nodeord')
            routeid = item.findAll('routeid')  
            updowncd = item.findAll('updowncd')
            
            if updowncd == []:
                updowncd = None
            elif updowncd[0].text == '0':
                updowncd = '상행'
            elif updowncd[0].text == '1':
                updowncd = '하행'
            elif updowncd[0].text == '2':
                updowncd = '2'

            sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd, citycode) VALUES (?,?,?,?,?,?,?,?,?)"
            val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, nodeno, nodeord[0].text, routeid[0].text, updowncd, location[0])

            cursor.execute(sql, val)
            mydb.commit()

조회날짜 : 20211008 | location : 35320 | cnt : 1 | start TSB313000010 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 2 | start TSB313000011 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 3 | start TSB313000012 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 4 | start TSB313000014 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 5 | start TSB313000018 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 6 | start TSB313000019 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 7 | start TSB313000020 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 8 | start TSB313000021 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 9 | start TSB313000023 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 10 | start TSB313000024 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 11 | start TSB313000025 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt 

조회날짜 : 20211008 | location : 35320 | cnt : 95 | start TSB313000065 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 96 | start TSB313000066 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 97 | start TSB313000071 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 98 | start TSB313000075 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 99 | start TSB313000096 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 100 | start TSB313000097 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 101 | start TSB313000107 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 102 | start TSB313000114 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 103 | start TSB313000004 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 104 | start TSB313000005 public_busroute_info
조회날짜 : 20211008 | location : 35320 | cnt : 105 | start TSB313000008 public_busroute_info
조회날짜 : 20211008 | location

조회날짜 : 20211008 | location : 35340 | cnt : 5 | start TSB315000006 public_busroute_info
조회날짜 : 20211008 | location : 35340 | cnt : 6 | start TSB315000009 public_busroute_info
조회날짜 : 20211008 | location : 35340 | cnt : 7 | start TSB315000011 public_busroute_info
조회날짜 : 20211008 | location : 35340 | cnt : 8 | start TSB315000012 public_busroute_info
조회날짜 : 20211008 | location : 35340 | cnt : 9 | start TSB315000013 public_busroute_info
조회날짜 : 20211008 | location : 35340 | cnt : 10 | start TSB315000014 public_busroute_info
조회날짜 : 20211008 | location : 35340 | cnt : 11 | start TSB315000015 public_busroute_info
조회날짜 : 20211008 | location : 35340 | cnt : 12 | start TSB315000017 public_busroute_info
조회날짜 : 20211008 | location : 35340 | cnt : 13 | start TSB315000020 public_busroute_info
조회날짜 : 20211008 | location : 35340 | cnt : 14 | start TSB315000021 public_busroute_info
조회날짜 : 20211008 | location : 35340 | cnt : 15 | start TSB315000022 public_busroute_info
조회날짜 : 20211008 | location : 35340 | 

조회날짜 : 20211008 | location : 35370 | cnt : 29 | start TSB318000052 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 30 | start TSB318000053 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 31 | start TSB318000054 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 32 | start TSB318000055 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 33 | start TSB318000057 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 34 | start TSB318000058 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 35 | start TSB318000061 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 36 | start TSB318000063 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 37 | start TSB318000066 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 38 | start TSB318000067 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 39 | start TSB318000069 public_busroute_info
조회날짜 : 20211008 | location : 353

조회날짜 : 20211008 | location : 35370 | cnt : 122 | start TSB318000428 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 123 | start TSB318000435 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 124 | start TSB318000436 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 125 | start TSB318000438 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 126 | start TSB318000439 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 127 | start TSB318000441 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 128 | start TSB318000442 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 129 | start TSB318000444 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 130 | start TSB318000446 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 131 | start TSB318000447 public_busroute_info
조회날짜 : 20211008 | location : 35370 | cnt : 132 | start TSB318000449 public_busroute_info
조회날짜 : 20211008 | loc

...

조회날짜 : 20211008 | location : 35380 | cnt : 89 | start TSB319000080 public_busroute_info
조회날짜 : 20211008 | location : 35380 | cnt : 90 | start TSB319000081 public_busroute_info
조회날짜 : 20211008 | location : 35380 | cnt : 91 | start TSB319000082 public_busroute_info
조회날짜 : 20211008 | location : 35380 | cnt : 92 | start TSB319000083 public_busroute_info
조회날짜 : 20211008 | location : 35380 | cnt : 93 | start TSB319000084 public_busroute_info
조회날짜 : 20211008 | location : 35380 | cnt : 94 | start TSB319000097 public_busroute_info
조회날짜 : 20211008 | location : 35380 | cnt : 95 | start TSB319000098 public_busroute_info
조회날짜 : 20211008 | location : 35380 | cnt : 96 | start TSB319000115 public_busroute_info
조회날짜 : 20211008 | location : 35380 | cnt : 97 | start TSB319000116 public_busroute_info
조회날짜 : 20211008 | location : 35380 | cnt : 98 | start TSB319000117 public_busroute_info
조회날짜 : 20211008 | location : 35380 | cnt : 99 | start TSB319000119 public_busroute_info
조회날짜 : 20211008 | location : 353

조회날짜 : 20211008 | location : 36010 | cnt : 20 | start MPB324000006 public_busroute_info
조회날짜 : 20211008 | location : 36010 | cnt : 21 | start MPB324000026 public_busroute_info
조회날짜 : 20211008 | location : 36010 | cnt : 22 | start MPB324000031 public_busroute_info
조회날짜 : 20211008 | location : 36010 | cnt : 23 | start MPB324000038 public_busroute_info
조회날짜 : 20211008 | location : 36010 | cnt : 24 | start MPB324000123 public_busroute_info
조회날짜 : 20211008 | location : 36010 | cnt : 25 | start MPB324000021 public_busroute_info
조회날짜 : 20211008 | location : 36010 | cnt : 26 | start MPB324000107 public_busroute_info
조회날짜 : 20211008 | location : 36010 | cnt : 27 | start MPB324000008 public_busroute_info
조회날짜 : 20211008 | location : 36010 | cnt : 28 | start MPB324000216 public_busroute_info
조회날짜 : 20211008 | location : 36010 | cnt : 29 | start MPB324000117 public_busroute_info
조회날짜 : 20211008 | location : 36010 | cnt : 30 | start MPB324000118 public_busroute_info
조회날짜 : 20211008 | location : 360

조회날짜 : 20211008 | location : 36020 | cnt : 83 | start YEB325000023 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 84 | start YEB325000024 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 85 | start YEB325000029 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 86 | start YEB325000030 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 87 | start YEB325000031 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 88 | start YEB325000036 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 89 | start YEB325000037 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 90 | start YEB325000039 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 91 | start YEB325000040 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 92 | start YEB325000041 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 93 | start YEB325000042 public_busroute_info
조회날짜 : 20211008 | location : 360

조회날짜 : 20211008 | location : 36020 | cnt : 176 | start YEB325000180 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 177 | start YEB325000181 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 178 | start YEB325000084 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 179 | start YEB325000093 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 180 | start YEB325000094 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 181 | start YEB325000095 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 182 | start YEB325000096 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 183 | start YEB325000199 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 184 | start YEB325000200 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 185 | start YEB325000224 public_busroute_info
조회날짜 : 20211008 | location : 36020 | cnt : 186 | start YEB325000230 public_busroute_info
조회날짜 : 20211008 | loc

조회날짜 : 20211008 | location : 36030 | cnt : 79 | start SCB326000126 public_busroute_info
조회날짜 : 20211008 | location : 36030 | cnt : 80 | start SCB326000127 public_busroute_info
조회날짜 : 20211008 | location : 36030 | cnt : 81 | start SCB326000128 public_busroute_info
조회날짜 : 20211008 | location : 36030 | cnt : 82 | start SCB326000129 public_busroute_info
조회날짜 : 20211008 | location : 36030 | cnt : 83 | start SCB326000130 public_busroute_info
조회날짜 : 20211008 | location : 36030 | cnt : 84 | start SCB326000158 public_busroute_info
조회날짜 : 20211008 | location : 36030 | cnt : 85 | start SCB326000159 public_busroute_info
조회날짜 : 20211008 | location : 36030 | cnt : 86 | start SCB326000228 public_busroute_info
조회날짜 : 20211008 | location : 36030 | cnt : 87 | start SCB326000229 public_busroute_info
조회날짜 : 20211008 | location : 36030 | cnt : 88 | start SCB326000240 public_busroute_info
조회날짜 : 20211008 | location : 36030 | cnt : 89 | start SCB326000241 public_busroute_info
조회날짜 : 20211008 | location : 360